In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Download NLTK resources (do it once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jayen\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
file=r'C:\Users\jayen\Text-sentiment-analysis-general-purpose\artifacts\Mental_health.csv'
df=pd.read_csv(file)
df

,text,label
0,RT @SchudioTv: Want to know more about #autism...,anxiety
1,We blame ourselves and feel worse. Start with ...,anxiety
2,"RT @PsychiatristCNS: 130,000 patient years and...",anxiety
3,RT @SkypeTherapist: See a therapist online ove...,anxiety
4,"RT @PsychiatristCNS: 130,000 patient years and...",anxiety
...,...,...
102054,"""Nothing beats the feeling of achieving someth...",normal
102055,Too much homework Feeling Stressed!,depression
102056,"overworked in school,, causes me headache",depression
102057,How come my friends immediately get to know st...,normal


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102059 entries, 0 to 102058
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    102046 non-null  object
 1   label   102056 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


In [14]:
df['label'].value_counts().head(10)

label
normal        29620
depression    20291
good          16837
suicide        9865
stress         8903
anxious        8388
lonely         6788
anxiety        1364
Name: count, dtype: int64

In [6]:
df.isnull().sum()

text     13
label     3
dtype: int64

In [9]:
df=df.dropna()

In [8]:
df.isnull().sum()

text     13
label     3
dtype: int64

In [5]:
pd.api.types.is_numeric_dtype(df['label'])

False

In [15]:
num_classes = len(set(df['label']))
num_classes

9

In [9]:
#NOW ILL DO SOME PREPROCESSING STEPS FOR THAT ILL CREATE A FUNCTION
def preprocessing(q):

  q=str(q).lower().strip()#strip gives a string in return if u use split it will give a list of words
  #The strip() method in Python is used to remove leading and trailing characters (whitespace characters by default) from a string before " Hello, world! " after"Hello, world!"
   # Replace certain special characters with their string equivalents
  q = q.replace('%', 'percent')
  q = q.replace('$', 'dollar ')
  q = q.replace('₹', 'rupee ')
  q = q.replace('€', 'euro ')
  q = q.replace('@', 'at')
  q=q.replace('&','and')

  q = re.sub(r'([0-9]+)000000000', r'\1b', re.sub(r'([0-9]+)000000', r'\1m', re.sub(r'([0-9]+)000', r'\1k', q)))#Input: "1500000000 Output: "1.5b" it will do this
  q=re.sub(r'\b(?:\d{1,4}[/-]\d{1,2}[/-]\d{1,4}|\d{1,2}[/-]\d{1,2}[/-]\d{2,4})\b','',q)#for removing dates
  q=re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b','',q)#removing email
  q= re.sub(r'https?://\S+|www\.\S+','',q)#removing urls
  q=re.sub("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF"
                             "\U0001F700-\U0001F77F\U0001F780-\U0001F7FF\U0001F800-\U0001F8FF"
                             "\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF"
                             "\U00002702-\U000027B0\U000024C2-\U0001F251]+", '', q)#this is for removing  emojis



    # https://en.wikipedia.org/wiki/Wikipedia%3aList_of_English_contractions
    # https://stackoverflow.com/a/19794953
  contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "im": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }
  q_decontracted=[]

  q_decontracted = [contractions[i] if i in contractions else i for i in q.split()]


  #now i join this word into q
  q=' '.join(q_decontracted)
  q = q.replace("'ve", " have")
  q = q.replace("n't", " not")
  q = q.replace("'re", " are")
  q = q.replace("'ll", " will")
  # Removing HTML tags
  q = BeautifulSoup(q)
  q = q.get_text()
  # Remove punctuations like ,.""[]
  q =re.sub(r'[^\w\s{}\/<>\'"\-_=+()*~:;]', ' ', q)

  q = word_tokenize(q)  # Convert to lowercase

  # Remove stopwords and non-alphabetic characters, perform lemmatization
  q = [
        lemmatizer.lemmatize(word) for word in q
        if word not in stop_words
    ]



  return ' '.join(q)#ALWAYS REMEMBER IN ORDER TO USE WORD2VEC UR SENTENSES SHOULD BE IN TOKKENS OR WORDS BUT IN THIS CASE WE HAVE TO TRAIN OUR MODEL WE HAVE TO TOOKENIZE
  #THE WORDS SO WE USE JOIN THEN AFTER WE TOKKENIZE IT WONT BE ABLE TO CONCATINATE LIST OF STRINGS IMPORTANT. contain lists instead of strings.


In [10]:
df['preprocessed']=df['text'].apply(preprocessing)


C:\Users\jayen\AppData\Local\Temp\ipykernel_13936\3071036855.py:158: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  q = BeautifulSoup(q)
C:\Users\jayen\AppData\Local\Temp\ipykernel_13936\378374874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['preprocessed']=df['text'].apply(preprocessing)


In [9]:
from sklearn.preprocessing import LabelEncoder
df['encodedlabel']=LabelEncoder().fit_transform(df['label'])

In [12]:
df

,text,label,encodedlabel
0,RT @SchudioTv: Want to know more about #autism...,anxiety,0
1,We blame ourselves and feel worse. Start with ...,anxiety,0
2,"RT @PsychiatristCNS: 130,000 patient years and...",anxiety,0
3,RT @SkypeTherapist: See a therapist online ove...,anxiety,0
4,"RT @PsychiatristCNS: 130,000 patient years and...",anxiety,0
...,...,...,...
102054,"""Nothing beats the feeling of achieving someth...",normal,5
102055,Too much homework Feeling Stressed!,depression,2
102056,"overworked in school,, causes me headache",depression,2
102057,How come my friends immediately get to know st...,normal,5


In [26]:
df['encodedlabel'].value_counts()

encodedlabel
5    29613
2    20288
3    16837
7     9865
6     8903
1     8388
4     6788
0     1364
Name: count, dtype: int64

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
# Create a CountVectorizer instance
vectorizer = CountVectorizer()

x=np.array(df['preprocessed'])
y=np.array(df['encodedlabel'])

X=vectorizer.fit_transform(x)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [24]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
v=xgb.XGBClassifier(objective='multi:softmax', num_class=len(set(y)))#objective='multi:softmax' indicates that the model should perform multiclass classification, and num_class=len(set(y)) sets the number of classes based on the unique labels in your target variable (y).
v.fit(X_train,y_train)
# Calculate accuracy
y_pred=v.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of XgboostClassifier: {accuracy * 100:.2f}%")

Accuracy of XgboostClassifier: 66.65%


now doing this with deep learning methods bidirectional LSTM

In [27]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Bidirectional
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['preprocessed'])
sequences = tokenizer.texts_to_sequences(df['preprocessed'])

In [28]:
max_length = 200  # Define your maximum sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length)#THIS WILL BE MY ACTUAL DATA THAT'LL TRAIN TEST AND SPLIT

In [30]:
word_index = tokenizer.word_index#THESE ARE MY UNIQUE WORDS WITH ITS UNIQUE INDEX
num_words = len(word_index)+1

In [32]:

## Creating model
from tensorflow.keras.layers import Dropout
embedding_vector_features=200 ##features representation each word wil get convert into a vector of 300
model=Sequential()
model.add(Embedding(num_words,embedding_vector_features,input_length=max_length))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(50,return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(50)))
model.add(Dense(8,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

import numpy as np
X_final=np.array(padded_sequences)
y_final=np.array(df['encodedlabel'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 200)          21221600  
                                                                 
 dropout_2 (Dropout)         (None, 200, 200)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 200, 100)          100400    
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 200, 100)          0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 100)               60400     
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 8)                

In [33]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='accuracy', mode='max', verbose=1, baseline=0.95)

In [34]:
model.fit(X_final,y_final, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/10
2552/2552 [==============================] - 1525s 590ms/step - loss: 0.9140 - accuracy: 0.6482 - val_loss: 1.2151 - val_accuracy: 0.4553
Epoch 2/10
2552/2552 [==============================] - 1354s 531ms/step - loss: 0.5841 - accuracy: 0.7569 - val_loss: 1.1409 - val_accuracy: 0.5373
Epoch 3/10
2552/2552 [==============================] - 1263s 495ms/step - loss: 0.4332 - accuracy: 0.8097 - val_loss: 1.1156 - val_accuracy: 0.6013
Epoch 4/10
2552/2552 [==============================] - 1323s 518ms/step - loss: 0.3428 - accuracy: 0.8405 - val_loss: 1.3734 - val_accuracy: 0.5641
Epoch 5/10
  51/2552 [..............................] - ETA: 21:03 - loss: 0.2812 - accuracy: 0.8597

KeyboardInterrupt: 

: 